In [82]:
import pandas as pd

In [569]:
def tidyYarns():
	import pandas as pd

	# Read in data and concatenate into single dataframe
	pur_filename = "~/Desktop/Pendleton/CSVs/Stops-PurchasedYarns.csv"
	twist_filename = "~/Desktop/Pendleton/CSVs/Stops-TwistYarns.csv"
	dyed_filename = "~/Desktop/Pendleton/CSVs/Stops-DyedYarns.csv"
	spun_filename = "~/Desktop/Pendleton/CSVs/Stops-SpunYarns.csv"

	yarn_filename = [pur_filename, twist_filename, dyed_filename, spun_filename]
	yarns_list = []
	for yarn in yarn_filename:
		yarns_list.append(pd.read_csv(yarn))
	yarns_df = pd.concat(yarns_list)

	# Clean columns names
	yarns_df.columns = map(str.lower, yarns_df.columns)
	yarns_df = yarns_df.rename(columns = {'yarn#':'yarn'})

	# Select and order columns
	yarns_df = yarns_df[['yarn', 'type']]

	# Drop duplicate rows
	yarns_df = yarns_df.drop_duplicates(['type','yarn'], inplace = False)

	return yarns_df

In [571]:
#from tidyBom import  tidyBom
#from tidyYarns import tidyYarns

# Load functions and set to local variables
tidy_Bom = tidyBom() 
tidy_Yarns = tidyYarns()
tidy_BomYarns = pd.merge(tidy_Bom, tidy_Yarns, on = 'yarn', how = 'outer')


In [572]:
tidy_BomYarns

,millstyle,yarn,lbsperpiece,type
0,3642-23893-2,36E-001,10.2,Spun
1,7152-24159-1,36E-001,30.9,Spun
2,2961A23944-1,36E-001,26.6,Spun
3,3835-24239-1,36E-001,3.6,Spun
4,7152-24141-1,36E-001,16.5,Spun
5,3849F23827-1,36E-001,24.6,Spun
6,7152-22646-1,36E-001,10.7,Spun
7,1890-01801-17,36E-001,9.8,Spun
8,2968A21741-1,36E-001,0.3,Spun
9,2968A23817-1,36E-001,8.3,Spun


In [582]:
millstyles = tidy_BomYarns.groupby('millstyle')
totalLbs = (millstyles.agg({'lbsperpiece' : 'sum'})
		    		  .rename(columns = {"lbsperpiece" : "totallbsperpiece",
                                         "yarn" : "num_yarns"}))

millstylesyarns = tidy_BomYarns.groupby(['millstyle','type'])
sumtype = (millstylesyarns.agg({'lbsperpiece': 'sum',
							'yarn' : 'count'})
						   .rename(columns = {"lbsperpiece" : "sumyarntype",
                                             "yarn" : 'num_yarns'}))

joined_lbsType = pd.merge(totalLbs, sumtype, left_index = True, right_index = True).reset_index('type')

joined_lbsType = joined_lbsType.assign(perc_present = joined_lbsType.sumyarntype / joined_lbsType.totallbsperpiece)

In [583]:
joined_lbsType.head(15)

,type,totallbsperpiece,sumyarntype,num_yarns,perc_present
millstyle,,,,,
1001-22413-1,Spun,55.6,55.6,1,1.000000
1001-23131-1,Spun,55.2,55.2,3,1.000000
1001-23131-2,Dye,55.2,18.3,1,0.331522
1001-23131-2,Spun,55.2,36.9,2,0.668478
1001-23131-4,Spun,55.2,55.2,3,1.000000
1001-23935-2,Dye,55.3,8.9,1,0.160940
1001-23935-2,Spun,55.3,46.4,2,0.839060
1001-23935-3,Dye,55.3,8.9,1,0.160940
1001-23935-3,Spun,55.3,46.4,2,0.839060


In [374]:
joined_lbsType['perc_spun'] =  if x == 'Spun' else 0 for x in joined_lbsType['type']]

In [520]:
millstylesyarns = joined_lbsType.groupby(['millstyle','type'])
added_nums = (millstylesyarns.assign(num_spun = )

SyntaxError: unexpected EOF while parsing (<ipython-input-520-1843c0f2ac79>, line 2)

In [513]:
#millstylesyarns = joined_lbsType.groupby(['millstyle','type'])
#added_perc = 

#joined_lbsType = joined_lbsType.assign(perc_spun = lambda x: joined_lbsType.perc_present if x == 'Spun' else 0)

In [ ]:
joined_lbsType.loc[joined]

In [456]:
joined_lbsType.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
millstyle           5 non-null object
type                5 non-null object
totallbsperpiece    5 non-null float64
num_yarns           5 non-null int64
sumyarntype         5 non-null float64
perc_present        5 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 320.0+ bytes


In [488]:
joined_lbsType.iloc[0]['type']

'Spun'

In [486]:
joined_lbsType.head()

,millstyle,type,totallbsperpiece,num_yarns,sumyarntype,perc_present,perc_spun
0,1001-22413-1,Spun,55.6,1,55.6,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...
1,1001-23131-1,Spun,55.2,3,55.2,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...
2,1001-23131-2,Dye,55.2,3,18.3,0.331522,0
3,1001-23131-2,Spun,55.2,3,36.9,0.668478,0 1.000000 1 1.000000 2 0.331522 3 ...
4,1001-23131-4,Spun,55.2,3,55.2,1.000000,0 1.000000 1 1.000000 2 0.331522 3 ...


In [509]:
def joinedM365Stops():
	# Load libraries and functions
	import pandas as pandas
	from tidyM365 import tidyM365
	from tidyStops import tidyStops

	# Set returned dataframe to local variables
	tidy_M365 = tidyM365()
	tidy_stops = tidyStops()

	# Merge dataframes
	tidy_M365Stops = pd.merge(tidy_M365, tidy_stops, on = ['date', 'loom', 'range'], how = 'inner')

	# 
	groupby = tidy_M365Stops.groupby(['date', 'range', 'millstyle', 'loom'])
	joined_M365Stops = (groupby.agg({'ppm' : 'mean',
									'mpx' : 'sum',
									'yds' : 'sum',
									'combinedstops' : 'sum'})
							.rename(columns = {'ppm' : 'avgppm',
											'mpx' : 'totalpx',
											'yds' : 'totalyards',
											'combinedstops' : 'totalstops'}))
	joined_M365Stops = joined_M365Stops.assign(yardsperpx = joined_M365Stops['totalyards'] / joined_M365Stops['totalpx'])

	return joined_M365Stops.reset_index()


In [510]:
joined_M365Stops = joinedM365Stops()
joined_M365Stops.shape

(7279, 9)

In [508]:
joined_M365Stops.head()

,date,range,millstyle,loom,avgppm,totalpx,totalyards,totalstops,yardsperpx
0,2015-07-15,542,542A14742-3,18,350.0,484.0,561.0,164.0,1.159091
1,2015-07-15,542,542A18155-1,26,360.0,174.0,280.5,132.0,1.612069
2,2015-07-15,542,542A23589-7,19,360.0,524.0,561.0,102.0,1.070611
3,2015-07-15,542,542A23906-1,6,370.0,1062.0,841.5,396.0,0.792373
4,2015-07-15,641,641A21316-2,23,350.0,538.0,544.5,94.0,1.012082
